<h1><center>Leetcode 401 - ? </center></h1>

### 416 Partition Equal Subset Sum
- Use backtrack will exceed time limit
- Reduce the problem into `subset_sum(S, target)` where `target = sum(S) / 2`
- Subset sum 问题 Dynamic Programming `DP[l, s] = ` whether the first l numbers can have a subset sum of s
$$SS[L, s] = \begin{cases}
SS[L[:-1], s] if L[-1] > s \\
SS[L[:-1], s] \lor SS[L[:-1], s-L[-1]] else
\end{cases}$$

In [ ]:
class Solution:
    sol = False
    def print_matrix(self, matrix):
        for row in matrix:
            print(row)
        
    def canPartition(self, nums: List[int]) -> bool:
        n = len(nums)
    
        def backtrack(cur_sum_l, cur_sum_r, cursor):
            # base case
            if cursor == n:
                if cur_sum_l == cur_sum_r:
                    self.sol = True
                return
            # recursive case
            backtrack(cur_sum_l + nums[cursor], cur_sum_r, cursor+1)
            backtrack(cur_sum_l, cur_sum_r + nums[cursor], cursor+1)
        # Driver
        backtrack()
        return self.sol
            
    
    class Solution:
        def canPartition(self, nums: List[int]) -> bool:
            cache = dict()
            
            # RETURNS: whether the l-prefix of nums can have a subset sum of s
            # MODIFIES: store the result in the cache
            def DP(l, s):
                # Base case
                if (l, s) in cache: return cache[(l, s)]
                if s == 0: return True
                if s < 0: return False
                if l == 0: return False
                
                # Recursive case
                include = DP(l-1, s-nums[l-1])
                skip = DP(l-1, s)
                cache[(l, s)] = include or skip
                return cache[(l, s)]
            
            # Driver
            if sum(nums) % 2 == 1: return False
            else:
                return DP(len(nums), sum(nums) // 2)
            
            

### 417 Pacific Atlantic Water Flow
- Search from target to src
- Search from src to target
- Graph search pseudocode (均可以用于 cover graph)
    - DFS
    ```
    def dfs(v):
        if v is visited:
            return
        else:
            visit(v)
            for u in G.adj[v]:
                dfs(u)
    ```
    - BFS
    ```
    from collections import deque
    def bfs(G, s):
        Q = deque([s])
        visited[s] = true
        while Q:
            v = Q.popleft()
            do(v)
            for u in G.adj[v]:
                if visited[u] == false:
                    Q.append(v)
                    visited[v] = true
            
    ```

In [ ]:
from collections import deque
class Solution:  
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        m = len(heights)
        n = len(heights[0])
        
        # a global state
        visitN = self.makeMatrix(m, n, 0)
        
        # a helper local function to check if (r, c) is a valid row-col coordinate
        def inRange(r, c):
            return 0 <= r < m and 0 <= c < n
        
        # breadth first search
        def bfs(s, visited):
            Q = deque([s])
            visited.add(s)

            while Q:
                (r, c) = Q.popleft()
                visitN[r][c] = visitN[r][c] + 1
                # build new frontier
                up = (r-1, c)
                down = (r+1, c)
                left = (r, c-1)
                right = (r, c+1)
                for (newR, newC) in [up, down, left, right]:
                    if inRange(newR, newC) and (newR, newC) not in visited and heights[newR][newC] >= heights[r][c]:
                        Q.append((newR, newC))
                        visited.add((newR, newC))
        
        # Main
        pacific = []
        for r in range(m):
            for c in range(n):
                if r == 0 or c == 0:
                    pacific.append((r, c))
        visited_pacific = set()
        bfs(pacific, visited_pacific)
        
        atlantic = []
        for r in range(m):
            for c in range(n):
                if r == m-1 or c == n-1:
                    atlantic.append((r, c))
        visited_atlantic = set()
        bfs(atlantic, visited_atlantic)
        
        # build final result
        res = []
        for r in range(m):
            for c in range(n):
                if visitN[r][c] == 2:
                    res.append([r, c])
        return res  

### 424 Longest Repeating Character Replacement
 - $Sliding window$
 - Rephrase the problem `modify at most k letters to be uniform` = `len - maxFreq <= k`
 - 'Only confined to uppercase/lowercase letters' $\implies$ Loop over all symbols costs O(1)
 - Keep track of the `maximally occurring letter's frequency` is usually O(1)
 - $monotonic$ property: substring strictly easier than superstring
    - 对于同一个 `end`, 若当前window invalid, 则无法通过将`begin`向左expand使得window变得valid.

In [ ]:
def characterReplacement(self, s: str, k: int) -> int:
    begin, end = 0, 0
    freq, maxFreq = dict(), 0
    # freq[c] = freq of c in the current window
    # maxFreq = the max_c(freq[c]) for c in the current window
    width = 0
    
    while end < len(s):
        # ----- 1. move end to right by one
        endChar = s[end]
        end += 1
        freq[endChar] = freq.get(endChar, 0) + 1
        maxFreq = max(maxFreq, freq[endChar])
        
        # ----- 2. keep moving begin to right to restore the property
        #------ always able to restore it before begin > end
        while end - begin - maxFreq > k:
            beginChar = s[begin]
            begin += 1
            freq[beginChar] -= 1
            # update the max properties
            # -> this takes only O(1) time
            maxFreq = 0
            for c in string.ascii_uppercase:
                maxFreq = max(maxFreq, freq.get(c, 0))
        
        # ----- 3. now the window [begin, end) is restored 
        width = max(width, end - begin)
                
    return width

### 435. Non-overlapping Intervals
 - For any pair of two intervals, if they are overlapping, at least one of them must be removed from the original array of intervals `=>` iterate over the array in pair of two `A` and `B`
 - Mindset: At each iteration of the loop, we have a prefix of the original array that we have traversed so far, and we choose to keep a `subsequence` of that `prefix`.
 - `A`: the rightmost interval of the `subsequence`
 - `B`: the interval in the original array that is currently being processed
 - Between `A` and `B`, keep the one with the smaller end value and delete the one with a larger end value and move on to the next `A', B'`

In [ ]:
class Solution:
    def eraseOverlapIntervals(self, intervals: List[List[int]]) -> int:
        # Handle degenerate case
        if len(intervals) < 2: return 0
        
        # Handle normal case
        start = 0
        end = 1
        
        intervals.sort()
        
        A = intervals[0] 
        index = 1
        removed = 0
        
        # A: the rightmost interval that is temporarily kept in the final array
        # B: the iterator in the original array
        while index < len(intervals):
            B = intervals[index]
            # if A and B are not overlapping
            if A[end] <= B[start]:
                A = B
                index += 1
            # if A and B are overlapping
            else:
                removed += 1
                # remove A and keep B
                if A[end] > B[end]:
                    A = B
                    index += 1
                # remove B and keep A
                else:
                    index += 1
        
        return removed
        

### 438 Find All Anagrams in a String
- `Sliding Window Technique` 使用场景 <br>
    Generally speaking, the sliding window technique is useful when you need to keep track of a `contiguous sequence of elements`, such as summing up the values in a subarray.
- 不要忘记 `sliding window` 和 `dict` data structure 常常可以一起用
- Anagram 问题 当明确 `all letters are lowercase / English letters` 的时候, comparing and updating the datastructure takes $$O(26) = O(1)$$ times
- Anagram 的问题 不要忘记使用 `dict` data structure
- When using `defaultdict` data structure on non-`int` types, init with `defaultdict(lambda: 0)`

In [ ]:
import string
from collections import defaultdict

class Solution:
    def same_dict(self, dict1, dict2):
        for c in string.ascii_lowercase:
            if dict1[c] != dict2[c]:
                return False
        return True
    
    def findAnagrams(self, s: str, p: str) -> List[int]:
        # handle degenrate casee
        if len(s) < len(p):
            return []
        
        # handle normal cases
        p_cnt = defaultdict(lambda: 0)
        s_cnt = defaultdict(lambda: 0)
        
        # init pattern dict
        for char in p:
            p_cnt[char] += 1
        
        # traverse s with a sliding window [left, right) of length |p|
        left = 0
        right = len(p)
        for i in range(left, right):
            s_cnt[s[i]] += 1
        
        ans = []
        while right <= len(s):
            # consider the current window
            if self.same_dict(p_cnt, s_cnt):
                ans.append(left)
            if right == len(s): 
                break
            # construct the next window
            s_cnt[s[left]] -= 1
            s_cnt[s[right]] += 1
            # move to the next window
            left += 1
            right += 1
        
        return ans

### 449 Serialize and Deserialize BST
- Preorder is a sufficient linear representation for any tree (can find the root)

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Codec:
    def numsToStr(self, nums):
        s = ""
        for num in nums:
            s = s + "#"
            s = s + str(num)
        return s[1:]
        
    def strToNums(self, s):
        l = s.split('#')
        nums = []
        for s in l:
            nums.append(int(s))
        return nums
    
    def serialize(self, root: Optional[TreeNode]) -> str:
        """Encodes a tree to a single string.
        """
        # handle degenerate case
        if root is None:
            return ""
        # handle normal cases
        def preorder(root):
            # base case
            if root is None:
                return []
            # recursive case
            return [root.val] + preorder(root.left) + preorder(root.right)
            
        linear = preorder(root)
        
        return self.numsToStr(linear)
            
    def deserialize(self, data: str) -> Optional[TreeNode]:
        """Decodes your encoded data to tree.
        """
        # handle degenerate case
        if len(data) == 0:
            return None
        # handle normal cases
        def preorderToTree(l):
            # base case
            if len(l) == 0:
                return None
            if len(l) == 1:
                node = TreeNode(l[0])
                return node
            
            # recursive case
            # find the first index whose value is greater than the root
            i = 1
            while i < len(l) and l[i] < l[0]:
                i += 1
            left = l[1:i]
            right = l[i:]
            node = TreeNode(l[0])
            node.left = preorderToTree(left)
            node.right = preorderToTree(right)
            return node
                
        l = self.strToNums(data)
        return preorderToTree(l)
    


### 454 4Sum II
 - Loosed condition: (1) only need to count ! + (2) only need indices to be unique !
 - Sol1: `Complements` + `Cache result in Dict` $O(n^2)$
 - Sol2: Generalized to `kSumCount`
 - Use recursion to simulate variable-depth nested loop e.g. `build_hash` and `cntComplements`

In [ ]:
def fourSumCount(self, nums1: List[int], nums2: List[int], nums3: List[int], nums4: List[int]) -> int:
    abFreq = defaultdict(int)

    for a in nums1:
        for b in nums2:
            abFreq[a+b] += 1

    res = 0
    for c in nums3:
        for d in nums4:
            res += abFreq[-(c+d)]
    return res

def kSumCount(self, lists, target):
    k = len(lists)
    
    # This function returns all 
    # freq[] : sum(a1,a2,...,ak) -> occurence
    # pairs
    freq = defaultdict(int)
    
    def buildHash(cursor, total):
        # Base case
        if cursor == k // 2: # already the kth innermost loop
            freq[total] += 1
        # Recursive case
        else:
            for num in lists[cursor]:
                buildHash(cursor+1, total+num)
    
    def cntComplements(cursor, total):
        # Base case
        if cursor == k:
            complement = target - total
            return freq[complement]
        # Recursive case
        else:
            res = 0
            for num in lists[cursor]:
                res += cntComplements(cursor+1, total+num)
            return res
    # Driver function
    buildHash(0, 0)
    return cntComplements(k // 2, 0)

### 487 Max Consecutive Ones II
 - Standard sliding window
 - Keep track of the count of `0`s in the current window
 - Monotonic Property.

In [ ]:
def findMaxConsecutiveOnes(self, nums: List[int]) -> int:
    left, right = 0, 0
    lastZeroIdx = None
    zeroCnt = 0
    width = 0
    
    while right < len(nums):
        # ----- 1. move right by one
        rightNum = nums[right]
        right += 1
        if rightNum == 0:
            zeroCnt += 1
        
        # ----- 2. keep moving left to restore the window
        # guaranteed to terminate as left < right
        while zeroCnt > 1:
            leftNum = nums[left]
            if leftNum == 0:
                zeroCnt -= 1
            left += 1
                
        # ----- 3. Update answer wrt this window
        width = max(width, right - left)
    
    return width

### 494 Target Sum
- Backtrack will cause time limit exceed. `Backtrack` only helps to organize the search process to be exhaustive + non-redundant, but it does not save any work.
- Use DP when there can be redundant computations

In [ ]:
class backtrack:
    ans = 0
    def findTargetSumWays(self, nums: List[int], target: int) -> int:
        n = len(nums)
        
        def backtrack(cursor, res):
            # base case
            if cursor == n:
                if res == target:
                    self.ans += 1
                return
            # recursive case
            # option 1 add +
            backtrack(cursor+1, res+nums[cursor])
            # option 2
            backtrack(cursor+1, res-nums[cursor])
        
        # driver
        backtrack(0, 0)
        return self.ans

### 543 Diameter of Binary Tree
- Key observation: longest tree in a tree must look like for some node `v` in the tree, 
$$dist(v) = v \text{ to a leaf in } v.left +  v \text{ to a leaf in } v.right$$
- Write a recursive function to compute `dist(v)` by recursing it at all the `v` in the tree.
- Update a global variable `diameter` to concurrently takes the max of them (don't need to write them down, update `diameter` every time we calculate the value and about to return it) 等价于
$$ ans = \max_{v \in T} dist(v) $$
but was performed concurrently with the recursive calls
- A recursive function that
    - returns the depth of this node `to be used by its parent caller`
    - imperatively modifies the `global var diameter` with `dist(v)` for all `v in T`

In [ ]:
class Solution:
    diameter = 0
    def diameterOfBinaryTree(self, root: Optional[TreeNode]) -> int:
        # RETURNS: the depth to be used by its parent caller
        # MODIFIES: keep updating the 'diameter' with the path length using current root as the 'peak'
        def return_depth_modifies_path(root):
            # base case 
            if root is None:
                # 1. MODIFIES:
                self.diameter = max(self.diameter, 0)
                # 2. RETURNS:
                return 0
            
            # recursive case
            # 1. MODIFIES
            left_depth = return_depth_modifies_path(root.left)
            right_depth = return_depth_modifies_path(root.right)
            self.diameter = max(self.diameter, left_depth + right_depth) # no need to +1 because we count edges
            # 2. RETURNS
            return 1 + max(left_depth, right_depth)
        
        # driver
        return_depth_modifies_path(root)
        return self.diameter

### 559 Maximum Depth of N-ary Tree
 - Push `(node, depth)` as entries in the stack

In [ ]:
def maxDepth(self, root: 'Node') -> int:
    if root is None: return 0
    '''
    else:
        maxChlidDepty = 0
        for chld in root.children:
            maxChlidDepty = max(maxChlidDepty, self.maxDepth(chld))
        return maxChlidDepty + 1
    '''
    stack = [(root, 1)]
    maxDep = 1
    while stack:
        node, dep = stack.pop()
        maxDep = max(maxDep, dep)
        for child in node.children:
            if child:
                stack.append((child, dep + 1))
    return maxDep

### 560 Subarray Sum Equals K
- 思路 consider each index `j` as the ending position of a candidate subarray, see how many `i` comes before it such that `[i .. j]` is a valid subarray
- `Prefix Sum` 算法 <br>
    $\blacksquare$  subarray的问题中, 一个subarray往往可以用两个prefixes去解决, 由于prefixes只有O(n)个 $\implies$ allows us to solve the probelm in $O(n)$ <br>
    $\blacksquare$  generate the array of prefix sum `ps` with O(1) cost之后的用法 <br>
        - subarray $sum(i, j) = ps[j] - ps[i-1]$ <br>
        - `[i .. j]` sums to `k` $\iff$ `[0 .. i-1]` sums to `ps[i]-k` <br>
- The `hashmap K-sum subarray algorithm`
    - Thread 1: updating a dict `prefix sum value` $\to$ `number of prefixes with this sum`
    - Thread 2: update a variable `counter` at the same time of updating the `dict`
    - Pseudocode 1: This function generates a hashmap `prefix sum value` $\to$ `number of prefixes with this sum`
        ```
        def generate_hash_map(A):
            D = defaultdict()
            sum = 0
            D[sum] = 1

            for a in A:
                sum = sum + a
                D[sum] += 1
            return D
        ```
    - Pseudocode 2: From the hashmap we can count the number of subarrays that sum to `k` <br>
        - considering current i as the ending index of a subarray, 
        - count how many j's before it such that `[j ... i]` sums to `k`
        - which is equiv to how many prefixes [0 ... j-1] before it sums to `sum-k`
        - 
        ```
        def count_k_subarray(A):
            D = generate_hash_map(A)
            sum = 0
            count = 0

            for a in A
                # Example
                # [0, 1, 2, 3, ..., j-1] [j, ..., i]
                #    sum - k              k
                sum += a
                count += D[sum-k]
            return count
        ```

In [ ]:
from collections import defaultdict
def subarraySum(self, nums: List[int], k: int) -> int:
    ans = 0
    prefsum = 0
    d = defaultdict()
    d[0] = 1
    for num in nums:
        prefsum = prefsum +num
        ans += d[prefsum-k]
        d[prefsum] += 1
    return ans

### 567 Permutation in String
 - Keep window of length exactly k ` + ` Bad symbols zeroed ` ===> ` the current window is a permutation
 - 思路 Use $O(1)$ cost to transition from the previous window to the current window
 - Sliding window
 - Validation criteria: 
    - `badSymbols = len(stillNeeded)`
    - When adding letter from the right
    ```
    if rightChar in stillNeeded:
        stillNeeded[rightChar] -= 1
        if stillNeeded[rightChar] == 0:
            badSymbols -= 1
    ```
    - When removing letter from the left
    ```
    if leftChar in stillNeeded:
        stillNeeded[leftChar] += 1
        if stillNeeded[leftChar] == 1:
            badSymbols += 1      
    ```

In [ ]:
class Solution:
    def checkInclusion(self, s1: str, s2: str) -> bool:
        k = len(s1)
        n = len(s2)
        stillNeeded = collections.Counter(s1)
        left, right = 0, 0
        badSymbols = len(stillNeeded)  # to check if the current symbol is valid or not in O(1)
        # actually iterating over string.ascii_lowercase is also O(1) time
        
        # print(stillNeeded)
        # window is confined to be the length of s1
        while right < len(s2):
            # ----- 1. move right by one
            rightChar = s2[right]
            if rightChar in stillNeeded:
                stillNeeded[rightChar] -= 1
                if stillNeeded[rightChar] == 0:
                    badSymbols -= 1
            right += 1
        
            # ----- 2. remove the rightmost one
            if right - left == k:
                if badSymbols == 0: 
                    return True
                leftChar = s2[left]
                left += 1
                if leftChar in stillNeeded:
                    stillNeeded[leftChar] += 1
                    if stillNeeded[leftChar] == 1:
                        badSymbols += 1           
            # print(left, right, s2[left:right], badSymbols)
        return False
            
            

### 572 Subtree of Another Tree
- Solution One: recursive tree traversal
- `haveStarted` 用于记录是否已经开始 Match subtree 的过程, 一旦开始 Match 就不能停止
且左右两子树的 `haveStarted` 必须一致
- `treeMatch(root, subRoot, haveStarted)`中`haveStarted`表示在root之前是否开始matching

- Solution Two: Encode tree inorder traversal as a string (delimiters: `#`)

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def treeMatch(self, root, subRoot, haveStarted):
        # Base Case
        if root is None and subRoot is None:
            return True
        if root is None:
            return False
        if subRoot is None:
            return not haveStarted
            
        # Recursive Case
        if haveStarted:
            return root.val==subRoot.val and self.treeMatch(root.left, subRoot.left, True) and self.treeMatch(root.right, subRoot.right, True)
        else:
            if root.val == subRoot.val:
                # options 1: start matching now
                matchNow = self.treeMatch(root.left, subRoot.left, True) and self.treeMatch(root.right, subRoot.right, True)
                # option 2: suspend matching
                matchLater = self.treeMatch(root.left, subRoot, False) or self.treeMatch(root.right, subRoot, False)
                return matchNow or matchLater
            else:
                # only: suspend matching for this node and push the choice to left and right subtree
                matchLater = self.treeMatch(root.left, subRoot, False) or self.treeMatch(root.right, subRoot, False)
                return matchLater
                
        
    def isSubtree(self, root: Optional[TreeNode], subRoot: Optional[TreeNode]) -> bool:
        return self.treeMatch(root, subRoot, False) or self.treeMatch(root, subRoot, True)
            
        

### 617 Merge Two Binary Trees
- Easy recursion

In [ ]:
# Definition for a binary tree node.
 class TreeNode:
     def __init__(self, val=0, left=None, right=None):
         self.val = val
         self.left = left
         self.right = right
class Solution:
    def mergeTrees(self, root1: Optional[TreeNode], root2: Optional[TreeNode]) -> Optional[TreeNode]:
        # base case
        if root1 is None and root2 is None:
            return None
        elif root1 is None:
            return root2
        elif root2 is None:
            return root1
        
        # recursive case
        else:
            curr = TreeNode(root1.val + root2.val)
            curr.left = self.mergeTrees(root1.left, root2.left)
            curr.right = self.mergeTrees(root1.right, root2.right)
            return curr

### 647 Palindromic Substrings
 - Approach One: Buttom Up DP `DP[i, j] = whether s[i, j) is a palindrome`
 - Approcah Two: Expand around centers

In [ ]:
def countSubstrings(self, s: str) -> int:
        # DP[i][j]: if s[i:j) is a palindrome or not
        n = len(s)
        DP = self.makeMatrix(n+1, n+1, False)
        
        # Base Case
        for i in range(n):
            DP[i][i+1] = True # singleton letter is always a palindrome
        for i in range(n-1):
            DP[i][i+2] = (s[i] == s[i+1])
        
        # Recursive Case
        for l in range(3, n+1):
            for i in range(n-l+1):
                DP[i][i+l] = DP[i+1][i+l-1] and (s[i] == s[i+l-1])
        
        # Count the number of palindromes by length
        res = 0
        for l in range(1, n+1):
            for i in range(n-l+1):
                res += DP[i][i+l]
        return res

def countSubstrings(self, s: str) -> int:
        # A helper function computing the number of palindromes centering at s[lo], s[hi]
        def expandAroundCenter(lo, hi):
            count = 0
            while lo >= 0 and hi < len(s):
                if s[lo] != s[hi]: break
                count += 1    
                lo -= 1
                hi += 1
            return count
        
        # Main
        res = 0
        for i in range(len(s)):
            res += expandAroundCenter(i, i)
            res += expandAroundCenter(i, i + 1)
        return res

### 653 Two Sum IV - Input is a BST
 - 2SUM

In [ ]:
def findTarget(self, root: Optional[TreeNode], k: int) -> bool:
    freq = defaultdict(int)
    
    def search(root):
        if root is None:
            return
        else:
            search(root.left)
            freq[root.val] += 1
            search(root.right)
    
    search(root)
    
    for num1 in freq:
        num2 = k - num1
        if num1 == num2 and freq[num1] >= 2:
            return True
        if num1 != num2 and num2 in freq:
            return True
    return False

### 695 Max Area of Island
 - Define `bfs` in a clever way
    - Returns the area beginning from the `[row][col]`
    - Modifies all the cells visited along the way to prevent future visit

In [ ]:
class Solution:
    def maxAreaOfIsland(self, grid: List[List[int]]) -> int:
        
        m = len(grid)
        n = len(grid[0])
        
        def bfs(row, col):
            # base case 
            if row < 0 or row >= m or col < 0 or col >= n: return 0
            if grid[row][col] != 1: return 0
            # recursive case
            area = 1
            grid[row][col] = -1
            for (drow, dcol) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                area += bfs(row + drow, col + dcol)
            return area
        
        maxArea = 0
        
        for row in range(m):
            for col in range(n):
                if grid[row][col] == 1:
                    maxArea = max(maxArea, bfs(row, col))
        return maxArea

### 703 Kth Largest Element in a Stream
 - Keep track of a min heap of size `k` where the top is the smallest element in the largest `k` elements scanned so far
 - `kth largest elements` -> only the `smallest` among the largest `k` elements need to be compared with the incoming `val`

In [ ]:
import math
import heapq
class KthLargest:

    def __init__(self, k: int, nums: List[int]):
            self.k = k
            self.heap = []       # keeping track of the k-largest numbers
            heapq.heapify(self.heap)
            
            for num in nums: self.add(num)

    def add(self, val: int) -> int:
        
        if len(self.heap) < self.k:
            heapq.heappush(self.heap, (val, val))
            _, topVal = self.heap[0]
            return topVal
        else:
            _, topVal = self.heap[0] # the smallest value in the heap right now
            if val < topVal: # disregard the incoming value
                return topVal
            else:
                heapq.heappop(self.heap)
                heapq.heappush(self.heap, (val, val))
                _, newTopVal = self.heap[0]
                return newTopVal
                
        raise Exception("cannot be here")
    

### 236 Lowest Common Ancestor of a Binary Tree
 - Use property: the stack returns from the base case call to the original function call
 - `if n == p or n == q: return n` $\implies$ a node is found
 - Execution flow: whenever a node `p` or `q` is found, that node is returned:
   - case 1:  LCA $\in$ `{p, q}` : By property of the stack, whoever is higher in the tree will be returned later in the function call stack $\implies$ whoever between `p` and `q` is higher will overwrite the other one $\implies$ the LCA is correctly returned
   - case 2: LCA $\notin$ `{p, q}` This will be detecetd for the first time by `if left_flag and right_flag: return n`. From then on,`n` is always to its caller until the original function call/

### 704 Binary Search
 - Simple binary search
 

In [ ]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        lo, hi = 0, len(nums)
        
        while lo < hi:
            mid = lo + (hi - lo) // 2
            if nums[mid] == target: return mid
            elif nums[mid] > target: hi = mid
            else: lo = mid + 1
        
        return -1
                

### 739 Daily Temperatures
- `Monotonic Stack` <br>
> Monotonic stacks help us maintain minimum and maximum element WITHIN A RANGE and keeps the sorted order of elements within this range <br>
 ====> So that to include an incoming element, we do not need to compare it against all elements within this range to (1) get the new minima and maxima and (2) keep the stack sorted
- Cost: each entry is pushed into and poped from the stack at most once, so the cost is `O(2n) = O(n)`
- Enumerate a list when we also care about indices
`for i, x in enumerate(L):`

In [ ]:
class Solution:
    def dailyTemperatures(self, temperatures: List[int]) -> List[int]:
        n = len(temperatures)
        ans = [0] * n
        stack = []
        
        # inv: the stack currently keep track of days that haven't found a warmer future day yet
        # inv: the days and temperatures in the stack are sorted in non-decreasing order
        for cur_day, cur_temp in enumerate(temperatures):
            while stack and temperatures[stack[-1]] < cur_temp:
                prev_day = stack[-1]
                # now that prev_day finds cur_day as its closest warmer day in future
                ans[prev_day] = cur_day - prev_day
                stack.pop()
            stack.append(cur_day)
        
        return ans

### 746 Min Cost Climbing Stairs
 - DP `DP[i] =` the min cost paied to step on stair `[i]` (do not need to pay for `[i]`)

In [ ]:
class Solution:
    def minCostClimbingStairs(self, cost: List[int]) -> int:
        n = len(cost)
        DP = [math.inf] * (n+1) # DP[i] = the min cost to reach [i]
        
        # Base case
        DP[0] = 0
        DP[1] = 0
        
        # Recursive case
        for i in range(2, n+1):
            DP[i] = min(
                DP[i-1] + cost[i-1], # start from [i-1] and climb 1 step
                DP[i-2] + cost[i-2], # start from [i-2] and climb 2 steps
            )
        
        print(DP)
        return DP[n]

### 763 Partition Labels
- Use `dict` to record the last occuring index of each `symbol` in `s`
- Every time, start with a `left` and use a while loop to derive the `right` by repeatedly updating `right = max(right, last[c])`
- Using a sliding window technique, the three steps should be organized as:
    - 1. record `right`,  move `hi` and update the data structure to reflect this move
    - 2. `while (window not valid)`, remove `lo` to the right, preserve the data structure
    - 3. now `[left, right)` is again a new valid window, do something about it

In [ ]:
from collections import defaultdict
class Solution:
    def partitionLabels(self, s: str) -> List[int]:
        last_index = defaultdict(int)
        for i, c in enumerate(s):
            last_index[c] = i
        
        parts = []      # holding the length all the parts
        
        index = 0
        while index < len(s):
            earlist_end = last_index[s[index]]
            part = [ s[index] ]
            
            while index < earlist_end:
                c = s[index]
                part.append(c)
                index += 1
                earlist_end = max(earlist_end, last_index[c])
                
            parts.append(len(part))
            index += 1
            
        return parts
        
            

### 846. Hand of Straights
 - The intervals are greedily identified by the starting element (which is the smallest element remaining so far)
 - (1) count the occurance of each element as a dict
 - (2) Repeat for the intended number of rounds:
 - (3) First traverse the unique numbers `starts` from smallest to largest, stop at the first number whose remaining occurance `>=1` => Build the interval from this number

In [ ]:
class Solution:
    def isNStraightHand(self, hand: List[int], groupSize: int) -> bool:
        # Handle degenerate case
        if len(hand) % groupSize != 0: return False
        
        # Handle normal cases
        freq = collections.Counter(hand)
        starts = sorted(freq.keys())
        starts_idx = 0
        start = starts[starts_idx]
        
        for _ in range(len(hand) // groupSize):
            while freq[start] <= 0: 
                starts_idx += 1
                start = starts[starts_idx]
                
            for num in range(start, start + groupSize):
                print(num)
                if freq[num] <= 0: return False
                freq[num] -= 1
        
        return True
        

### 875 Koko Eating Bananas
 - Boundary finding Binary Search
     - Keep the boundray to be inclusive on both ends 
 - Pseudo code for boundary finding binary search to find i such that
    - Case 1 `N N N N N N [Y] Y Y Y Y Y Y` (find the leftmost valid)
      ```
      def binsearch_leftmost():
         lo, hi = lowest possible, highest possible
         while lo < hi:
            mid = lo + (hi - lo) // 2
            if VALID(mid): hi = mid
            else: lo = mid + 1
         
         if VALID(lo): return "found"
         else: return "no such value"
      ```
    - Case 2 `Y Y Y Y Y [Y] N N N N N N N` (find the rightmost valid)
      ```
      def bin_search():
         lo, hi = lowest possible, highest possible
         while lo < hi:
            mid = lo + (hi - lo) // 2
            if VALID (mid): lo = mid
            else: hi = mid - 1
         if VALID(lo): return "found"
         else: return "no such value"
      ```

In [ ]:
class Solution:
    def minEatingSpeed(self, piles: List[int], h: int) -> int:
        lo, hi = 1, max(piles)
        
        def speed_enough(speed):
            time = 0
            for pile in piles:
                time += ceil(pile / speed)
            return time <= h
        
        # inv: the possible speed lies within [lo, hi] both inclusive
        while lo < hi:
            mid = lo + (hi-lo) // 2
            if speed_enough(mid):
                hi = mid
            else:
                lo = mid + 1
            
        if speed_enough(lo): return lo
        else: raise Exception("cannot reach here")

### 973 K Closest Points to Origin
 - Keep track of a min heap of the `k closest points` scanned so far
    - key: `-dist(x, y)`
    - val: `[x, y]`
 - Only need to compare the `farthest point of the k closest points` with the new incoming point

In [ ]:
class Solution:
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        def dist(x, y):
            return x * x + y * y
        
        kClosest = []
        heapq.heapify(kClosest)
        
        for [x, y] in points:
            if len(kClosest) < k:
                heapq.heappush(kClosest, (-dist(x, y), [x, y]))
            else:
                farthestDist, farthestPoint = heapq.heappop(kClosest)
                farthestDist *= -1
                
                newDist = dist(x, y)
                if newDist >=farthestDist: 
                    heapq.heappush(kClosest, (-farthestDist, farthestPoint))
                else:
                    heapq.heappush(kClosest, (-newDist, [x, y]))
        
        return [point for (dist, point) in kClosest]

### 994 Rotting Oranges
 - BFS
    - `from` all the rotten oranges as the initial frontier
    - `via` can only go through fresh oranges `gridvalue == 1`
    - `to` until the frontier is empty
    - Perform last check over the entire grid for any fresh orange that cannot be reached by rotten organges

In [ ]:
class Solution:
    def orangesRotting(self, grid: List[List[int]]) -> int:
        # the longest manhattan distance
        m = len(grid)
        n = len(grid[0])
        rots = []
        
        for row in range(m):
            for col in range(n):
                if grid[row][col] == 2:
                    rots.append((row, col))
        
        Q = deque(rots)
        
        distance = 0
        while True:
            distance += 1
            frontier_size = len(Q)

            for _ in range(frontier_size):
                (row, col) = Q.popleft()
                for (drow, dcol) in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
                    newrow = row + drow
                    newcol = col + dcol
                    if (0 <= newrow < m) and (0 <= newcol < n) and (grid[newrow][newcol] == 1):
                        grid[newrow][newcol] = 2
                        Q.append((newrow, newcol))
            
            if len(Q) == 0: break
        
        # Search the grid again for existance of fresh orange
        for row in grid:
            for orange in row:
                if orange == 1: 
                    return -1
        
        return distance - 1

### 1046 Last Stone Weight

In [ ]:
class Solution:
    def lastStoneWeight(self, stones: List[int]) -> int:
        stones = [-s for s in stones]
        
        while len(stones) >= 2:
            x = -heapq.heappop(stones)
            y = -heapq.heappop(stones)
            
            if x > y: x, y = y, x
            
            if x == y: continue
            if x != y: heapq.heappush(stones, -(y - x))
        
        if len(stones) == 1: 
            weight = -heapq.heappop(stones)
            return weight
        else:
            return 0

### 1099 Two Sum Less Than K
 - the array is sorted => `Two Pointer Method`

In [ ]:
def twoSumLessThanK(self, nums: List[int], k: int) -> int:
    nums.sort()
    n = len(nums)
    lo, hi = 0, n-1
    
    best = -1
    
    while lo < hi:
        curSum = nums[lo] + nums[hi]
        if curSum < k:
            lo += 1
            best = max(best, curSum)
        else:
            hi -= 1
        
    return best

### 1143 Longest Common Subsequence

In [ ]:
class Solution:
    def createMatrix(self, rows, cols, init):
        return [ [init for _ in range(cols)] for _ in range(rows) ]
    
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        n1 = len(text1)
        n2 = len(text2)
        
        if n1 * n2 == 0: return 0
        
        DP = self.createMatrix(n1 + 1, n2 + 1, 0)
        
        for l1 in range(0, n1+1):
            DP[l1][0] = 0
        
        for l2 in range(0, n2+1):
            DP[0][l2] = 0
        
        for l1 in range(1, n1+1):
            for l2 in range(1, n2+1):
                if text1[l1-1] == text2[l2-1]:
                    DP[l1][l2] = 1 + DP[l1-1][l2-1]
                else:
                    DP[l1][l2] = max(DP[l1-1][l2], DP[l1][l2-1])
        
        return DP[n1][n2]

### 1448 Count Good Nodes in Binary Tree
 - Simple DFS 
 - Rephrase: `greater than all values from root to its parent` = `greater than the max value` = `keep track of the max ancestor is enough`

In [ ]:
import math
class Solution:
    def goodNodes(self, root: TreeNode) -> int:
        # Handle degenerate case
        if root is None: return 0
        
        # Handle normal cases
        goodNodes = [0]
        
        # [root]:  the current node being visited
        # [maxAncestor]: the max node value from root to the parent of root
        def search(root, maxAncestor):
            # Base case
            if root is None: return
            # Recursive case
            if root.val >= maxAncestor: goodNodes[0] += 1
            newMaxAncestor = max(maxAncestor, root.val)
            search(root.left, newMaxAncestor)
            search(root.right, newMaxAncestor)
        
        # Driver
        search(root, -math.inf)
        return goodNodes[0]

### 1679 Max Number of K-Sum Pairs
- For 'pair problems', each element `N` is uniquely pair with its complement `target-N`. All the other numbers are not meaningful to `N`.

In [ ]:
def maxOperations(self, nums: List[int], k: int) -> int:
    freq = dict()
    
    for n in nums:
        freq[n] = freq.get(n, 0) + 1
    
    pair_same = 0
    pair_diff = 0
    print(freq)
    
    for x in freq:
        # print(x)
        if x == k-x:
            pair_same += freq.get(x, 0) // 2
        else:
            a = freq.get(x, 0)
            b = freq.get(k-x, 0)
            pair_diff += min(a, b)
    
    return pair_same + pair_diff // 2